### 这个架构应该就是最终要实现的，到时候改一下 prompt 就行
### 另外要实现一些工具，就像search一样。

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [2]:

from qunar import react_chat_sql_json
prompt = react_chat_sql_json.chat_art_prompt_template
prompt1 = hub.pull("hwchase17/react-chat-json")
chat_model = ChatOpenAI(temperature=0, model='gpt-4')

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.utilities.serpapi import SerpAPIWrapper
from langchain.tools import Tool
from langchain.tools.render import render_text_description
from tools import  sql_runner

search = SerpAPIWrapper(serpapi_api_key = "bb32d37fbdca24717613e25dbda90e9b6e34a663c863c94d4c5c4fcadb089f55")
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="SQL Runner",
        func=sql_runner.run_sql,
        description="you can input a sql and get the url of the result, please insert the url into the final answer as a embedded html"
    )
]

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

chat_model_with_stop = chat_model.bind(stop=["\nObservation"])

from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_messages

# We need some extra steering, or the chat model forgets how to respond sometimes
TEMPLATE_TOOL_RESPONSE = """TOOL RESPONSE: 
---------------------
{observation}

USER'S INPUT
--------------------

Okay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"""

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_messages(x['intermediate_steps'], template_tool_response=TEMPLATE_TOOL_RESPONSE),
    "chat_history": lambda x: x["chat_history"],
} | prompt | chat_model_with_stop | JSONAgentOutputParser()

from langchain.agents import AgentExecutor

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory,handle_parsing_errors=True)


In [4]:
agent_executor.invoke({"input": "请生成各个房间的返后间夜量"})['output']



> Entering new AgentExecutor chain...


TypeError: argument of type 'NoneType' is not iterable

In [5]:
agent_executor.invoke({"input": "我指的是各个房间类型的返后间夜量"})['output']




> Entering new AgentExecutor chain...
Could not parse LLM output: 对不起，我需要更多的信息来理解您的问题。"返后间夜量"是一个特定的术语吗？如果是的话，能否请您解释一下这个术语的含义？这将有助于我为您提供更准确的SQL查询语句。Invalid or incomplete response```json
{
    "action": "Final Answer",
    "action_input": "对不起，我需要更多的信息来理解您的问题。\"返后间夜量\"是一个特定的术语吗？如果是的话，能否请您解释一下这个术语的含义？这将有助于我为您提供更准确的SQL查询语句。"
}
```

> Finished chain.


'对不起，我需要更多的信息来理解您的问题。"返后间夜量"是一个特定的术语吗？如果是的话，能否请您解释一下这个术语的含义？这将有助于我为您提供更准确的SQL查询语句。'

In [5]:
agent_executor.invoke({"input": "那就提供各个房间类型的间夜量吧"})['output']




> Entering new AgentExecutor chain...
```json
{
    "action": "SQL Runner",
    "action_input": "SELECT room_type, COUNT(*) as nights FROM bookings GROUP BY room_type"
}
```https://baidu.com/q?SELECT room_type, COUNT(*) as nights FROM bookings GROUP BY room_type```json
{
    "action": "Final Answer",
    "action_input": "I have executed the SQL query to get the number of nights for each room type. You can check the results [here](https://baidu.com/q?SELECT room_type, COUNT(*) as nights FROM bookings GROUP BY room_type)."
}
```

> Finished chain.


'I have executed the SQL query to get the number of nights for each room type. You can check the results [here](https://baidu.com/q?SELECT room_type, COUNT(*) as nights FROM bookings GROUP BY room_type).'

In [ ]:
prompt